<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Topic Modeling
## *Data Science Unit 4 Sprint 1 Assignment 4*

Analyze a corpus of Amazon reviews from Unit 4 Sprint 1 Module 1's lecture using topic modeling: 

- Fit a Gensim LDA topic model on Amazon Reviews
- Select appropriate number of topics
- Create some dope visualization of the topics
- Write a few bullets on your findings in markdown at the end
- **Note**: You don't *have* to use generators for this assignment

In [1]:
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, KFold, train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.ensemble import GradientBoostingClassifier
from scipy.stats import randint
from sklearn.naive_bayes import MultinomialNB
from sklearn.decomposition import TruncatedSVD

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

import spacy
nlp = spacy.load("en_core_web_lg")

import pandas as pd
import numpy as np
import re

#other imports for testing
import unidecode
from word2number import w2n

#ntlk imports
import nltk.corpus
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')

#tokenizing
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize

#stemming
from nltk.stem import PorterStemmer

#lemmitizing
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

#modeling
import lightgbm as lgb
from lightgbm import LGBMClassifier

import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora
import gensim.downloader as api
from gensim.models.ldamulticore import LdaMulticore
from gensim.models import CoherenceModel
from gensim.parsing.preprocessing import preprocess_string, strip_tags, strip_punctuation, strip_numeric, stem_text
from gensim.parsing.preprocessing import strip_multiple_whitespaces, strip_non_alphanum, remove_stopwords, strip_short

import pprint as pprint
import os
import pyLDAvis
import pyLDAvis.gensim 
import matplotlib.pyplot as plt

%matplotlib inline
pd.set_option('display.max_colwidth', 0)
pd.options.display.float_format = '{:.5f}'.format

import ipywidgets as widgets
import warnings
warnings.filterwarnings("ignore")

/opt/anaconda3/envs/U4-S1-NLP/lib/python3.8/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
/opt/anaconda3/envs/U4-S1-NLP/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:17: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.9 it will stop working
  from collections import Mapping, defaultdict
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/codyreece/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/codyreece/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/codyreece/nltk_data...
[nltk_data]   Package wordnet is already up-to

In [2]:
df = pd.read_csv('./data/imbd_keywords.csv')
df.head(2)

,review,sentiment,keywords
0,"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.I would say the main appeal of the show is due to the fact that it goes where other shows wouldn't dare. Forget pretty pictures painted for mainstream audiences, forget charm, forget romance...OZ doesn't mess around. The first episode I ever saw struck me as so nasty it was surreal, I couldn't say I was ready for it, but as I watched more, I developed a taste for Oz, and got accustomed to the high levels of graphic violence. Not just violence, but injustice (crooked guards who'll be sold out for a nickel, inmates who'll kill on order and get away with it, well mannered, middle class inmates being turned into prison bitches due to their lack of street skills or prison experience) Watching Oz, you may become comfortable with what is uncomfortable viewing....thats if you can get in touch with your darker side.",positive,"['other shows', 'graphic violence', 'prison experience', 'prison bitches', 'violence', 'emerald city', 'dodgy dealings', 'glass fronts', 'inmates', 'street skills', 'unflinching scenes', 'crooked guards', 'the oswald maximum security state penitentary', 'mainstream audiences', 'oz', 'city', 'shady agreements', 'pretty pictures', 'touch', 'muslims', 'irish', 'italians', 'gangstas', 'latinos', 'christians', 'home', 'first', 'charm', 'the high levels', 'drugs', 'right', 'regards', 'well mannered, middle class inmates', 'sex', 'a show', 'scuffles', 'order', 'aryans', 'just 1 oz episode', 'the first episode', 'the other reviewers', 'your darker side', 'the prison', 'the main appeal', 'romance', 'the first thing', 'an experimental section', 'privacy', 'the classic use', 'the word', 'no punches', 'its brutality', 'their lack', 'a taste', 'the fact', 'the nickname', 'the agenda', 'all the cells', 'a nickel', 'you']"
1,"A wonderful little production. The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. The actors are extremely well chosen- Michael Sheen not only ""has got all the polari"" but he has all the voices down pat too! You can truly see the seamless editing guided by the references to Williams' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. A masterful production about one of the great master's of comedy and his life. The realism really comes home with the little things: the fantasy of the guard which, rather than use the traditional 'dream' techniques remains solid then disappears. It plays on our knowledge and our senses, particularly with the scenes concerning Orton and Halliwell and the sets (particularly of their flat with Halliwell's murals decorating every surface) are terribly well done.",positive,"['halliwell', 'michael sheen', 'realism', 'comedy', 'williams', 'piece', 'bbc', 'orton', 'a wonderful little production', 'polari', 'pat', 'halliwells murals', 'williams diary entries', 'the little things', 'the seamless editing', 'the filming technique', 'the entire piece', 'the great master', 'a masterful production', 'very unas

In [3]:
df = pd.DataFrame({
    'content': df['review'],
    'target': df['sentiment'],
    'keywords': df['keywords']
})
print(df.head(1))

In [4]:
# For reference on regex: https://docs.python.org/3/library/re.html
# From 'content' column: 

# 1. Remove new line characters
df['clean_text'] = df['content'].apply(lambda x: re.sub('\s+', ' ', x))

# 2. Remove Emails
df['clean_text'] = df['clean_text'].apply(lambda x: re.sub('From: \S+@\S+', '', x))

# 3. Remove non-alphanumeric characters
df['clean_text'] = df['clean_text'].apply(lambda x: re.sub('[^a-zA-Z]', ' ', x))

# 4. Remove extra whitespace 
df['clean_text'] = df['clean_text'].apply(lambda x: ' '.join(x.split()))

In [5]:
df.head(1)

,content,target,keywords,clean_text
0,"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.I would say the main appeal of the show is due to the fact that it goes where other shows wouldn't dare. Forget pretty pictures painted for mainstream audiences, forget charm, forget romance...OZ doesn't mess around. The first episode I ever saw struck me as so nasty it was surreal, I couldn't say I was ready for it, but as I watched more, I developed a taste for Oz, and got accustomed to the high levels of graphic violence. Not just violence, but injustice (crooked guards who'll be sold out for a nickel, inmates who'll kill on order and get away with it, well mannered, middle class inmates being turned into prison bitches due to their lack of street skills or prison experience) Watching Oz, you may become comfortable with what is uncomfortable viewing....thats if you can get in touch with your darker side.",positive,"['other shows', 'graphic violence', 'prison experience', 'prison bitches', 'violence', 'emerald city', 'dodgy dealings', 'glass fronts', 'inmates', 'street skills', 'unflinching scenes', 'crooked guards', 'the oswald maximum security state penitentary', 'mainstream audiences', 'oz', 'city', 'shady agreements', 'pretty pictures', 'touch', 'muslims', 'irish', 'italians', 'gangstas', 'latinos', 'christians', 'home', 'first', 'charm', 'the high levels', 'drugs', 'right', 'regards', 'well mannered, middle class inmates', 'sex', 'a show', 'scuffles', 'order', 'aryans', 'just 1 oz episode', 'the first episode', 'the other reviewers', 'your darker side', 'the prison', 'the main appeal', 'romance', 'the first thing', 'an experimental section', 'privacy', 'the classic use', 'the word', 'no punches', 'its brutality', 'their lack', 'a taste', 'the fact', 'the nickname', 'the agenda', 'all the cells', 'a nickel', 'you']",One of the other reviewers has mentioned that after watching just Oz episode you ll be hooked They are right as this is exactly what happened with me The first thing that struck me about Oz was its brutality and unflinching scenes of violence which set in right from the word GO Trust me this is not a show for the faint hearted or timid This show pulls no punches with regards to drugs sex or violence Its is hardcore in the classic use of the word It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary It focuses mainly on Emerald City an experimental section of the prison where all the cells have glass fronts and face inwards so privacy is not high on the agenda Em City is home to many Aryans Muslims gangstas Latinos Christians Italians Irish and more so scuffles death stares dodgy dealings and shady agreements are never far away I would say the main appeal of the show is due to the fact that it goes where other shows wouldn t dare Forget pretty pictures painted for mainstream audiences forget charm forget romance OZ doesn t mess around The first episode I ever saw struck me as so nasty it was surreal I couldn t say I was ready for it but as I watched more I developed a taste for Oz and got accustomed to the high levels of graphic violence Not 

In [6]:
#Checking for the best number of workers
N_JOBS = [None, 1, 2, 3, 4, -1]
import time
for n_jobs in N_JOBS:
        start = time.perf_counter()
        print('n_jobs = {}, perf_counter = {}'.format(n_jobs, time.perf_counter() - start))

n_jobs = None, perf_counter = 2.7839999994228037e-06
n_jobs = 1, perf_counter = 5.480000009328023e-07
n_jobs = 2, perf_counter = 4.149999988101172e-07
n_jobs = 3, perf_counter = 2.3900000023502344e-07
n_jobs = 4, perf_counter = 4.0399999967632994e-07
n_jobs = -1, perf_counter = 3.7100000049861137e-07


In [7]:
from tqdm import tqdm
tqdm.pandas()

from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [8]:
# Create 'lemmas' column
def get_lemmas(x):
    lemmas = []
    for token in nlp(x):
        if (token.is_stop!=True) and (token.is_punct!=True):
            lemmas.append(token.lemma_)
    return lemmas

In [10]:
%%time
df['lemmas'] = df['content'].parallel_apply(get_lemmas)

KeyboardInterrupt: 

Process ForkPoolWorker-24:
Traceback (most recent call last):
  File "/opt/anaconda3/envs/U4-S1-NLP/lib/python3.8/multiprocessing/process.py", line 313, in _bootstrap
    self.run()
  File "/opt/anaconda3/envs/U4-S1-NLP/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/anaconda3/envs/U4-S1-NLP/lib/python3.8/multiprocessing/pool.py", line 125, in worker
    result = (True, func(*args, **kwds))
  File "/opt/anaconda3/envs/U4-S1-NLP/lib/python3.8/multiprocessing/pool.py", line 48, in mapstar
    return list(map(*args))
  File "/opt/anaconda3/envs/U4-S1-NLP/lib/python3.8/site-packages/pandarallel/pandarallel.py", line 64, in global_worker
    return _func(x)
  File "/opt/anaconda3/envs/U4-S1-NLP/lib/python3.8/site-packages/pandarallel/pandarallel.py", line 140, in wrapper
    result = function(
  File "/opt/anaconda3/envs/U4-S1-NLP/lib/python3.8/site-packages/pandarallel/data_types/series.py", line 20, in worker
    retur

  File "<ipython-input-8-77c55c3044b7>", line 7, in get_lemmas
    return lemmas
  File "/opt/anaconda3/envs/U4-S1-NLP/lib/python3.8/site-packages/spacy/language.py", line 449, in __call__
    doc = proc(doc, **component_cfg.get(name, {}))
  File "nn_parser.pyx", line 274, in spacy.syntax.nn_parser.Parser.predict
  File "/opt/anaconda3/envs/U4-S1-NLP/lib/python3.8/site-packages/spacy/language.py", line 449, in __call__
    doc = proc(doc, **component_cfg.get(name, {}))
  File "/opt/anaconda3/envs/U4-S1-NLP/lib/python3.8/site-packages/spacy/language.py", line 449, in __call__
    doc = proc(doc, **component_cfg.get(name, {}))
  File "nn_parser.pyx", line 287, in spacy.syntax.nn_parser.Parser.greedy_parse
  File "/opt/anaconda3/envs/U4-S1-NLP/lib/python3.8/site-packages/thinc/neural/_classes/model.py", line 167, in __call__
    return self.predict(x)
Traceback (most recent call last):
  File "/opt/anaconda3/envs/U4-S1-NLP/lib/python3.8/site-packages/thinc/neural/_classes/model.py", line 

  File "/opt/anaconda3/envs/U4-S1-NLP/lib/python3.8/site-packages/thinc/neural/_classes/model.py", line 167, in __call__
    return self.predict(x)
  File "_parser_model.pyx", line 243, in spacy.syntax._parser_model.ParserModel.begin_update
  File "/opt/anaconda3/envs/U4-S1-NLP/lib/python3.8/site-packages/thinc/neural/_classes/model.py", line 167, in __call__
    return self.predict(x)
  File "/opt/anaconda3/envs/U4-S1-NLP/lib/python3.8/site-packages/thinc/neural/_classes/resnet.py", line 18, in predict
    Y = self._layers[0](X)
  File "_parser_model.pyx", line 293, in spacy.syntax._parser_model.ParserStepModel.__init__
  File "/opt/anaconda3/envs/U4-S1-NLP/lib/python3.8/site-packages/thinc/neural/_classes/model.py", line 131, in predict
    y, _ = self.begin_update(X, drop=None)
  File "/opt/anaconda3/envs/U4-S1-NLP/lib/python3.8/site-packages/thinc/neural/_classes/model.py", line 167, in __call__
    return self.predict(x)
  File "/opt/anaconda3/envs/U4-S1-NLP/lib/python3.8/site-pac

## Stretch Goals

* Incorporate Named Entity Recognition in your analysis
* Incorporate some custom pre-processing from our previous lessons (like spacy lemmatization)
* Analyze a dataset of interest to you with topic modeling